In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt  # Fixed: 'matplotlib as plt' → 'matplotlib.pyplot as plt'

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler  # Fixed: 'StandarScaler' → 'StandardScaler'
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

path="/home/dan/PLATZI/data/SKlearn_Orchestration/skLearn_Orchestration/data/third_part/"
file="heart.csv"
df=pd.read_csv(path+file)

In [2]:
df_features=df.drop(["target"],axis=1)
df_features.sample(4)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
61,66,0,2,146,278,0,0,152,0,0.0,1,1,2
476,57,1,0,165,289,1,0,124,0,1.0,1,3,3
238,60,1,0,117,230,1,1,160,1,1.4,2,2,3
190,41,1,2,112,250,0,1,179,0,0.0,2,0,2


In [3]:
df_target=df["target"]

In [4]:
#normalization
df_features=StandardScaler().fit_transform(df_features)
#split
X_train,X_test,Y_train,Y_test=train_test_split(df_features,df_target,test_size=0.25,random_state=37)


In [8]:
pca=PCA(n_components=4)
pca.fit(X_train)
ipca=IncrementalPCA(n_components=4,batch_size=300)#for low resorces in your pc
ipca.fit(X_train)
pc1_weights=pd.Series(pca.components_[0], index=df.drop("target", axis=1).columns).abs().sort_values(ascending=False)
print(f"here the variance of the components{pca.explained_variance_ratio_}")
print(f"now the ones with incremental pca{ipca.explained_variance_ratio_}")
pc1_weights


here the variance of the components[0.21419059 0.12436593 0.0958789  0.08669756]
now the ones with incremental pca[0.21393898 0.12389329 0.0937706  0.08294379]


thalach     0.431794
oldpeak     0.402505
exang       0.379148
slope       0.373866
age         0.319731
cp          0.301202
ca          0.248313
thal        0.200290
trestbps    0.169481
chol        0.141641
restecg     0.129245
sex         0.066264
fbs         0.058376
dtype: float64

the weights of the features on each pca axis come from the engeine vectores from the covariance matrix, now we can know how much a feature  is represented in the axis, now the ratios is wich porcentage of the variance is reprecented in the pca_ratio, with this we can know if the axis we  are using  are valuable compresions of our features

In [6]:
df_features.dtype

dtype('float64')

logistic regrassion

In [14]:
log=LogisticRegression(solver='lbfgs')
dt_train=pca.transform(X_train)
dt_test=pca.transform(X_test)
log.fit(dt_train,Y_train)
print(f"pca scores:{log.score(dt_test,Y_test)}")

log=LogisticRegression(solver='lbfgs')
dt_train=ipca.transform(X_train)
dt_test=ipca.transform(X_test)
log.fit(dt_train,Y_train)
print(f"pca scores:{log.score(dt_test,Y_test)}")

pca scores:0.8132295719844358
pca scores:0.8404669260700389


que es un solver? es un algoritmo para minimisar la funcion objetivo como gradient decent?

In [21]:
from sklearn.decomposition import KernelPCA
KPCA=KernelPCA(n_components=3,kernel='rbf')
KPCA.fit(X_train)
dt_train=KPCA.transform(X_train)
dt_test=KPCA.transform(X_test)
log=LogisticRegression(solver='lbfgs').fit(dt_train,Y_train)
print(f"Kpca scores:{log.score(dt_test,Y_test)}")


Kpca scores:0.8054474708171206
